# Лабораторная работа №4
## Задание:
1 . Провести классификацию найденного датасета, методами наивного Байеса  . В формате Markdown написать пояснения. Объяснить почему были выбраны именно такие гиперпараметры, была ли перекрестная проверка, и т.д.

## Решение:
#### Подключение библиотек


In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.naive_bayes import GaussianNB, BernoulliNB, MultinomialNB
from sklearn.metrics import accuracy_score, classification_report
from imblearn.under_sampling import RandomUnderSampler

#### Загрузка данных и вывод датасета


In [2]:
Dataset = pd.read_csv('cybersecurity_attacks.csv')  # Замени на путь к файлу
print(Dataset.head())

             Timestamp Source IP Address Destination IP Address  Source Port  \
0  2023-05-30 06:33:58     103.216.15.12           84.9.164.252        31225   
1  2020-08-26 07:08:30    78.199.217.198         66.191.137.154        17245   
2  2022-11-13 08:23:25      63.79.210.48          198.219.82.17        16811   
3  2023-07-02 10:38:46     163.42.196.10        101.228.192.255        20018   
4  2023-07-16 13:11:07     71.166.185.76        189.243.174.238         6131   

   Destination Port Protocol  Packet Length Packet Type Traffic Type  \
0             17616     ICMP            503        Data         HTTP   
1             48166     ICMP           1174        Data         HTTP   
2             53600      UDP            306     Control         HTTP   
3             32534      UDP            385        Data         HTTP   
4             26646      TCP           1462        Data          DNS   

                                        Payload Data  ... Action Taken  \
0  Qui natus

### Создание искомых столбец

In [3]:
label_encoder = LabelEncoder()
Dataset['Attack Type Encoded'] = label_encoder.fit_transform(Dataset['Attack Type'])
print(Dataset[['Attack Type', 'Attack Type Encoded']].head(10))

  Attack Type  Attack Type Encoded
0     Malware                    2
1     Malware                    2
2        DDoS                    0
3     Malware                    2
4        DDoS                    0
5     Malware                    2
6        DDoS                    0
7   Intrusion                    1
8   Intrusion                    1
9     Malware                    2


### Нормализация данных

In [4]:
non_numeric_columns = Dataset.select_dtypes(exclude=['number']).columns.tolist()
Dataset = Dataset.drop(columns=non_numeric_columns)

# Заполнение пустых значений средними значениями
columns_to_fill = ['Source Port', 'Destination Port', 'Packet Length', 'Anomaly Scores']
for column in columns_to_fill:
    mean_value = Dataset[column].mean()
    Dataset[column].fillna(mean_value, inplace=True)

print(Dataset)

       Source Port  Destination Port  Packet Length  Anomaly Scores  \
0            31225             17616            503           28.67   
1            17245             48166           1174           51.50   
2            16811             53600            306           87.42   
3            20018             32534            385           15.79   
4             6131             26646           1462            0.52   
...            ...               ...            ...             ...   
39319        31005              6764           1428           39.28   
39320         2553             28091           1184           27.25   
39321        22505             25152           1043           31.01   
39322        20013              2703            483           97.85   
39323        50137             55575           1175           34.63   

       Attack Type Encoded  
0                        2  
1                        2  
2                        0  
3                        2  
4 

### Поиск лучших метрик

In [5]:
X = Dataset.drop('Attack Type Encoded', axis=1)
y = Dataset['Attack Type Encoded']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=42)


under_sampler = RandomUnderSampler(random_state=42)


X_resampled, y_resampled = under_sampler.fit_resample(X_train, y_train)


# Определение методов наивного байеса
naive_bayes_models = {
    'GaussianNB': GaussianNB(),
    'MultinomialNB': MultinomialNB(),
    'BernoulliNB': BernoulliNB(),
}

# Параметры для GridSearchCV
param_grid = {
    'GaussianNB': {},
    'MultinomialNB': {'alpha': [0.1, 0.5, 1.0]},
    'BernoulliNB': {'alpha': [0.1, 0.5, 1.0], 'binarize': [0.0, 0.1, 0.2]},
}

# Выбор метрики для оценки моделей
scoring_metric = 'accuracy'

# Обучение и оценка моделей с использованием GridSearchCV
best_models = {}
for model_name, model in naive_bayes_models.items():
    grid_search = GridSearchCV(model, param_grid[model_name], scoring=scoring_metric, cv=5)
    grid_search.fit(X_resampled, y_resampled)

    best_models[model_name] = grid_search.best_estimator_
# Оценка наилучшей модели на тестовом наборе
best_model_name = max(best_models, key=lambda k: grid_search.cv_results_['mean_test_score'][grid_search.best_index_])
best_model = best_models[best_model_name]


# Вывод результатов
print(f"Лучшая модель: {best_model_name}")
print(f"Лучшие параметры: {grid_search.best_params_}")


Лучшая модель: GaussianNB
Лучшие параметры: {'alpha': 0.1, 'binarize': 0.2}


### обучение на лучших гиперпараметрах

In [6]:
# Retrain the best model on the entire dataset
best_model.fit(X_resampled, y_resampled)

# Make predictions on the entire dataset
y_pred_full = best_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred_full)
classification_rep = classification_report(y_test, y_pred_full)
print(f'Точность модели: {accuracy}')
print(classification_rep)

Точность модели: 0.3277813095994914
              precision    recall  f1-score   support

           0       0.32      0.32      0.32      5224
           1       0.32      0.26      0.29      5253
           2       0.33      0.40      0.37      5253

    accuracy                           0.33     15730
   macro avg       0.33      0.33      0.32     15730
weighted avg       0.33      0.33      0.32     15730

